In [1]:


# using Pkg
# Pkg.add("BenchmarkTools")
# Pkg.add("Distributions")
# Pkg.add("CxxWrap")
# Pkg.add("JLD")
# Pkg.add("ImageIO")
# Pkg.add("CircularArrays")
# Pkg.add("OffsetArrays")
# Pkg.add("TensorOperations")
# Pkg.add("TensorCast")
# Pkg.add("Tullio")
# Pkg.add("DiffEqOperators")
# Pkg.add("BandedMatrices")
# Pkg.add("PyPlot")
# Pkg.add("Plots")
# Pkg.add("FFTW")
# Pkg.add("SciPy")
# Pkg.add("GR")
# Pkg.add("Statistics")

import GR
using BenchmarkTools
using Distributions
using Random
using Plots
using PyCall
using PyPlot
#using Images
using CircularArrays
using CxxWrap
using FFTW
using SciPy
using JLD
using Statistics
stats = pyimport("scipy.stats")
np = pyimport("numpy")
cv2 = pyimport("cv2")



PyObject <module 'cv2' from '/usr/lib/python3/dist-packages/cv2.cpython-38-x86_64-linux-gnu.so'>

In [2]:
function Laplacian_3D!(P₁,P₂,B₁,B₂,Δx)
    A₁ = CircularArray(B₁);
    A₂ = CircularArray(B₂);
    Threads.@threads for 😄 ∈ CartesianIndices(P₁)
        (i,j,k) = Tuple(😄)
        P₁[i,j,k] = @fastmath (A₁[i+1,j,k] + A₁[i-1,j,k] + A₁[i,j + 1,k] + A₁[i,j - 1,k] + A₁[i,j,k + 1] + A₁[i,j,k-1] - 6A₁[i,j,k])/(Δx^2);
        P₂[i,j,k] = @fastmath (A₂[i+1,j,k] + A₂[i-1,j,k] + A₂[i,j + 1,k] + A₂[i,j - 1,k] + A₂[i,j,k + 1] + A₂[i,j,k-1] - 6A₂[i,j,k])/(Δx^2);
    end
    return nothing
end


function Cores_3D!(N,angle,thr)
    s = []
    count = 0
    accept = 0.5 - 0.5*thr/100
    for 😄 ∈ 1:(N-1)
        for 🥪 ∈ 1:(N-1)
            for 🎅 ∈ 1:(N-1)
                anorm₁ = (angle[😄,🥪,🎅] + π)/(2π)
                anorm₂ = (angle[😄+1,🥪,🎅] + π)/(2π)
                anorm₃ = (angle[😄+1,🥪+1,🎅] + π)/(2π)
                anorm₄ = (angle[😄,🥪+1,🎅] + π)/(2π)

                bnorm₁ = (angle[😄,🥪,🎅] + π)/(2π)
                bnorm₂ = (angle[😄+1,🥪,🎅] + π)/(2π)
                bnorm₃ = (angle[😄+1,🥪,🎅+1] + π)/(2π)
                bnorm₄ = (angle[😄,🥪,🎅+1] + π)/(2π)

                cnorm₁ = (angle[😄,🥪,🎅] + π)/(2π)
                cnorm₂ = (angle[😄,🥪+1,🎅] + π)/(2π)
                cnorm₃ = (angle[😄,🥪+1,🎅+1] + π)/(2π)
                cnorm₄ = (angle[😄,🥪,🎅+1] + π)/(2π)

                aθ₁ = min(abs(anorm₂ - anorm₁),1-abs(anorm₂ - anorm₁))
                aθ₂ = min(abs(anorm₃ - anorm₂),1-abs(anorm₃ - anorm₂))
                aθ₃ = min(abs(anorm₄ - anorm₃),1-abs(anorm₄ - anorm₃))

                bθ₁ = min(abs(bnorm₂ - bnorm₁),1-abs(bnorm₂ - bnorm₁))
                bθ₂ = min(abs(bnorm₃ - bnorm₂),1-abs(bnorm₃ - bnorm₂))
                bθ₃ = min(abs(bnorm₄ - bnorm₃),1-abs(bnorm₄ - bnorm₃))

                cθ₁ = min(abs(cnorm₂ - cnorm₁),1-abs(cnorm₂ - cnorm₁))
                cθ₂ = min(abs(cnorm₃ - cnorm₂),1-abs(cnorm₃ - cnorm₂))
                cθ₃ = min(abs(cnorm₄ - cnorm₃),1-abs(cnorm₄ - cnorm₃))

                aθ = aθ₁ + aθ₂ + aθ₃
                bθ = bθ₁ + bθ₂ + bθ₃
                cθ = cθ₁ + cθ₂ + cθ₃
                θₛ = aθ + bθ + cθ

                if θₛ > accept 
                    append!(s,[[😄,🥪,🎅]])
                end
            end
        end
    end

    if length(s) > 0
        for 🇸🇦 ∈ 1:(length(s)-1)
  
            diffᵣ = s[🇸🇦 + 1][1] - s[🇸🇦][1]
            diffₛ = s[🇸🇦 + 1][2] - s[🇸🇦][2]
            diffₜ = s[🇸🇦 + 1][3] - s[🇸🇦][3]

            if diffᵣ == 0 && diffₛ == 1 && diffₜ == 0
                count += 1
            end
            if diffᵣ == 1 && diffₛ == 0 && diffₜ == 0
                count += 1
            end
            if diffᵣ == 0 && diffₛ == 0 && diffₜ == 1
                count += 1
            end
        end
    end

    num = length(s) - count
    num = 1.5num

    return num,s
end




function setting!(moo)
    @inbounds Threads.@threads for t in eachindex(moo)
        if moo[t] > 1
            moo[t] = 1
        end
    end
    return nothing
end

function mooing!(moo,A₁,A₂)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(moo)
        (i,j,k) = Tuple(😄)
        @fastmath moo[i,j,k] = sqrt(A₁[i,j,k]^2 + A₂[i,j,k]^2)
    end
    return nothing
end


function angler!(angle,A₁,A₂)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(angle)
        (i,j,k) = Tuple(😄)
        angle[i,j,k] = @fastmath atan(A₂[i,j,k],A₁[i,j,k]);
    end
    return nothing
end

function meshgrid(xin,yin,zin)
    nx=length(xin)
    ny=length(yin)
    nz=length(zin)
    xout=zeros(nz,ny,nx)
    yout=zeros(nz,ny,nx)
    zout=zeros(nz,ny,nx)
    for jx=1:nx
        for ix=1:ny
            for hx=1:nz
                xout[hx,ix,jx]=xin[jx]
                yout[hx,ix,jx]=yin[ix]
                zout[hx,ix,jx]=zin[hx]
            end
        end
    end
    return (x=xout, y=yout, z=zout)
end

meshgrid (generic function with 1 method)

In [3]:
function PQfupdate_3D!(F₁,F₂,M₁,M₂,C₁,C₂,A₁,A₂,Ȧ₁,Ȧ₂,η,λ,fₐ)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(F₁)
        (i,j,k) = Tuple(😄)
        F₁[i,j,k] = @fastmath M₁[i,j,k] - λ * C₁[i,j,k] * η^2 * (A₁[i,j,k]^2 .+ A₂[i,j,k]^2 .- 1) - λ * C₁[i,j,k]*8.4e5 * 1e12/(3fₐ) - 2/η * Ȧ₁[i,j,k];
        F₂[i,j,k] = @fastmath M₂[i,j,k] - λ * C₂[i,j,k] * η^2 * (A₁[i,j,k]^2 .+ A₂[i,j,k]^2 .- 1) - λ * C₂[i,j,k]*8.4e5 * 1e12/(3fₐ) - 2/η * Ȧ₂[i,j,k];
    end
    return nothing
end


function PQAupdate_3D!(A₁,A₂,Δt,Ȧ₁,Ȧ₂,F₁,F₂)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(A₁)
        (i,j,k) = Tuple(😄)
        A₁[i,j,k] = @fastmath A₁[i,j,k] .+ Δt .* (Ȧ₁[i,j,k] .+ 0.5Δt .* F₁[i,j,k])
        A₂[i,j,k] = @fastmath A₂[i,j,k] .+ Δt .* (Ȧ₂[i,j,k] .+ 0.5Δt .* F₂[i,j,k])
    end
    return nothing
end


function PQvelupdate_3D!(Ȧ₁,Ȧ₂,Δt,F₁,F₂,M₁,M₂,C₁,C₂,A₁,A₂,η,λ,fₐ)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(Ȧ₁)
        (i,j,k) = Tuple(😄)
        Ȧ₁[i,j,k] = @fastmath Ȧ₁[i,j,k] .+ 0.5Δt .* (F₁[i,j,k] .+ M₁[i,j,k] - λ * C₁[i,j,k] * η^2 * (A₁[i,j,k]^2 .+ A₂[i,j,k]^2 .- 1) - λ * C₁[i,j,k]*8.4e5 * 1e12/(3fₐ) - 2/η * Ȧ₁[i,j,k]);
        Ȧ₂[i,j,k] = @fastmath Ȧ₂[i,j,k] .+ 0.5Δt .* (F₂[i,j,k] .+ M₂[i,j,k] - λ * C₂[i,j,k] * η^2 * (A₁[i,j,k]^2 .+ A₂[i,j,k]^2 .- 1) - λ * C₂[i,j,k]*8.4e5 * 1e12/(3fₐ) - 2/η * Ȧ₂[i,j,k]);
    end
    return nothing
end

function PQupdate_3D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt,time,fₐ)

    #F₁ .= M₁ .- a.^β .* λ .* A₁ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./time
    #F₂ .= M₂ .- a.^β .* λ .* A₂ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./time

    PQfupdate_3D!(F₁,F₂,M₁,M₂,A₁,A₂,A₁,A₂,Ȧ₁,Ȧ₂,time,1,fₐ)

    #A₁ .= A₁ .+ Δt .* (Ȧ₁ .+ 0.5Δt .* F₁)
    #A₂ .= A₂ .+ Δt .* (Ȧ₂ .+ 0.5Δt .* F₂)

    PQAupdate_3D!(A₁,A₂,Δt,Ȧ₁,Ȧ₂,F₁,F₂)

    Laplacian_3D!(M₁,M₂,A₁,A₂,Δx)

    #Ȧ₁ .= Ȧ₁ .+ 0.5Δt .* (F₁ .+ M₁ .- a₁.^β .* λ .* A₁ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./ (time + Δt))
    #Ȧ₂ .= Ȧ₂ .+ 0.5Δt .* (F₂ .+ M₂ .- a₁.^β .* λ .* A₂ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./ (time + Δt))

    
    PQvelupdate_3D!(Ȧ₁,Ȧ₂,Δt,F₁,F₂,M₁,M₂,A₁,A₂,A₁,A₂,time + Δt,1,fₐ)

    return nothing
end



PQupdate_3D! (generic function with 1 method)

In [4]:
function EQCDfupdate_3D!(F₁,F₂,M₁,M₂,C₁,C₂,A₁,A₂,Ȧ₁,Ȧ₂,η,λ,ηₓ,n)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(F₁)
        (i,j,k) = Tuple(😄)
        F₁[i,j,k] = @fastmath M₁[i,j,k] - λ * C₁[i,j,k] * η^2 * (A₁[i,j,k]^2 .+ A₂[i,j,k]^2 .- 1) + ηₓ^n *η^2 * C₂[i,j,k]^2/((A₁[i,j,k]^2 .+ A₂[i,j,k]^2)^1.5) - 2/η * Ȧ₁[i,j,k];
        F₂[i,j,k] = @fastmath M₂[i,j,k] - λ * C₂[i,j,k] * η^2 * (A₁[i,j,k]^2 .+ A₂[i,j,k]^2 .- 1) - ηₓ^n *η^2 * C₁[i,j,k]*C₂[i,j,k]/((A₁[i,j,k]^2 .+ A₂[i,j,k]^2)^1.5) - 2/η * Ȧ₂[i,j,k];
    end
    return nothing
end


function EQCDAupdate_3D!(A₁,A₂,Δt,Ȧ₁,Ȧ₂,F₁,F₂)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(A₁)
        (i,j,k) = Tuple(😄)
        A₁[i,j,k] = @fastmath A₁[i,j,k] .+ Δt .* (Ȧ₁[i,j,k] .+ 0.5Δt .* F₁[i,j,k])
        A₂[i,j,k] = @fastmath A₂[i,j,k] .+ Δt .* (Ȧ₂[i,j,k] .+ 0.5Δt .* F₂[i,j,k])
    end
    return nothing
end


function EQCDvelupdate_3D!(Ȧ₁,Ȧ₂,Δt,F₁,F₂,M₁,M₂,C₁,C₂,A₁,A₂,η,λ,ηₓ,n)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(Ȧ₁)
        (i,j,k) = Tuple(😄)
        Ȧ₁[i,j,k] = @fastmath Ȧ₁[i,j,k] .+ 0.5Δt .* (F₁[i,j,k] .+ M₁[i,j,k] - λ * C₁[i,j,k] * η^2 * (A₁[i,j,k]^2 .+ A₂[i,j,k]^2 .- 1) + ηₓ^n *η^2 * C₂[i,j,k]^2/((A₁[i,j,k]^2 .+ A₂[i,j,k]^2)^1.5) - 2/η * Ȧ₁[i,j,k])
        Ȧ₂[i,j,k] = @fastmath Ȧ₂[i,j,k] .+ 0.5Δt .* (F₂[i,j,k] .+ M₂[i,j,k] - λ * C₂[i,j,k] * η^2 * (A₁[i,j,k]^2 .+ A₂[i,j,k]^2 .- 1) - ηₓ^n *η^2 * C₁[i,j,k]*C₂[i,j,k]/((A₁[i,j,k]^2 .+ A₂[i,j,k]^2)^1.5) - 2/η * Ȧ₂[i,j,k])
    end
    return nothing
end


function EQCDupdate_3D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt,time,fₐ,r,s)

    n = 1

    #λ = (fₐ/mass(fₐ))^2;

    λᵪ = [1024 1448 3072 3584 5504];
    #ηₓ,η = ηtime(time,fₐ);
    λ = λᵪ[r]

    ηᵪ = range(2.8,stop = 3.6,length = 5)
    ηₓ = ηᵪ[s]

    if time < ηₓ
        ηₓ = time
    end
    

    #F₁ .= M₁ .- a.^β .* λ .* A₁ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./time
    #F₂ .= M₂ .- a.^β .* λ .* A₂ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./time

    EQCDfupdate_3D!(F₁,F₂,M₁,M₂,A₁,A₂,A₁,A₂,Ȧ₁,Ȧ₂,time,λ,ηₓ,n)

    #A₁ .= A₁ .+ Δt .* (Ȧ₁ .+ 0.5Δt .* F₁)
    #A₂ .= A₂ .+ Δt .* (Ȧ₂ .+ 0.5Δt .* F₂)

    EQCDAupdate_3D!(A₁,A₂,Δt,Ȧ₁,Ȧ₂,F₁,F₂)

    Laplacian_3D!(M₁,M₂,A₁,A₂,Δx)

    #Ȧ₁ .= Ȧ₁ .+ 0.5Δt .* (F₁ .+ M₁ .- a₁.^β .* λ .* A₁ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./ (time + Δt))
    #Ȧ₂ .= Ȧ₂ .+ 0.5Δt .* (F₂ .+ M₂ .- a₁.^β .* λ .* A₂ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./ (time + Δt))

    #ηₓ,η = ηtime(time+Δt,fₐ);

    EQCDvelupdate_3D!(Ȧ₁,Ȧ₂,Δt,F₁,F₂,M₁,M₂,A₁,A₂,A₁,A₂, time + Δt,λ,ηₓ,n)

    return nothing
end

EQCDupdate_3D! (generic function with 1 method)

In [5]:
function LLaplacian_3D!(P,A,Δx)
    B = CircularArray(A);
    Threads.@threads for 😄 ∈ CartesianIndices(P)
        (i,j,k) = Tuple(😄)
        P[i,j,k] = @fastmath (B[i+1,j,k] + B[i-1,j,k] + B[i,j + 1,k] + B[i,j - 1,k] + B[i,j,k + 1] + B[i,j,k-1] - 6B[i,j,k])/(Δx^2);
    end
    return nothing
end


function Lfupdate_3D!(F,M,A,Ȧ,η,ηₓ,n)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(F)
        (i,j,k) = Tuple(😄)
        F[i,j,k] = @fastmath M[i,j,k] - ηₓ^n * η^2 * sin(A[i,j,k]) - 2/η * Ȧ[i,j,k];
    end
    return nothing
end


function LAupdate_3D!(A,Δt,Ȧ,F)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(A)
        (i,j,k) = Tuple(😄)
        A[i,j,k] = @fastmath A[i,j,k] .+ Δt .* (Ȧ[i,j,k] .+ 0.5Δt .* F[i,j,k])
    end
    return nothing
end


function Lvelupdate_3D!(Ȧ,Δt,F,M,A,η,ηₓ,n)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(Ȧ)
        (i,j,k) = Tuple(😄)
        Ȧ[i,j,k] = @fastmath Ȧ[i,j,k] .+ 0.5Δt .* (F[i,j,k] .+ M[i,j,k] - ηₓ^n * η^2 * sin(A[i,j,k]) - 2/η * Ȧ[i,j,k])
    end
    return nothing
end


function Lupdate_3D!(A,Ȧ,M,F,Δx,Δt,time,fₐ,s)

    #ηₓ,η = Lηtime(time,fₐ);
    n = 1

    ηᵪ = range(2.8,stop = 3.6,length = 5)
    ηₓ = ηᵪ[s]

    if time < ηₓ
        ηₓ = time
    end

    #F₁ .= M₁ .- a.^β .* λ .* A₁ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./time
    #F₂ .= M₂ .- a.^β .* λ .* A₂ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./time

    Lfupdate_3D!(F,M,A,Ȧ,time,ηₓ,n)

    #A₁ .= A₁ .+ Δt .* (Ȧ₁ .+ 0.5Δt .* F₁)
    #A₂ .= A₂ .+ Δt .* (Ȧ₂ .+ 0.5Δt .* F₂)

    LAupdate_3D!(A,Δt,Ȧ,F)

    LLaplacian_3D!(M,A,Δx)

    #Ȧ₁ .= Ȧ₁ .+ 0.5Δt .* (F₁ .+ M₁ .- a₁.^β .* λ .* A₁ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./ (time + Δt))
    #Ȧ₂ .= Ȧ₂ .+ 0.5Δt .* (F₂ .+ M₂ .- a₁.^β .* λ .* A₂ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./ (time + Δt))

    #ηₓ,η = Lηtime(time+Δt,fₐ)

    Lvelupdate_3D!(Ȧ,Δt,F,M,A,time + Δt,ηₓ,n)

    return nothing
end


Lupdate_3D! (generic function with 1 method)

In [6]:
function FPQrun_3D!(n,N,t₀,A₁,A₂,Ȧ₁,Ȧ₂,Δx,Δt,fₐ,i,repeats)
    #700 images on per 5000 updates
    

    M₁ = zeros(N,N,N);
    M₂ = zeros(N,N,N);

    F₁ = zeros(N,N,N);
    F₂ = zeros(N,N,N);

    moo = zeros(N,N,N);
    Laplacian_3D!(M₁,M₂,A₁,A₂,Δx)

    angle = zeros(N,N,N);

    # fₛ = Δx
    # freq_s = fₛ/2 * range(-1,stop = 1,length = N)
    # freq = fₛ/2 * range(0,stop = 1, length = N/2)
    k_freq = fftfreq(N)*N
    kx,ky,kz = meshgrid(k_freq,k_freq,k_freq)

    knrm = sqrt.( kx.^2 + ky.^2 + kz.^2)
    knrm = collect(Iterators.flatten(knrm))

    kbins = range(0.5, N/2+1, step = 1)
    kvals = 0.5 * (kbins[2:end] + kbins[1:end-1])

    x = 500

    tracker = []
    cores = []
    thr = 1
    
    saxion = []
    axion = []

    axenergy = []
    
    lo = 0
    
    tim = t₀
    
    for 🥪 ∈ 1:repeats
        
        


        while tim < 200*🥪 + t₀
            tim = round(tim,digits = 4);


            if lo % x == 0
                #125 pictures
                mooing!(moo,A₁,A₂);
                #println(moo[1,1])
                #setting!(moo);
                angler!(angle,A₁,A₂);
                
    

                strng,loc = Cores_3D!(N,angle,thr)
                append!(tracker,tim)
                append!(cores,strng)
                
                loc₁ = []
                loc₂ = []
                loc₃ = []

                for i ∈ 1:length(loc)
                    append!(loc₁,loc[i][1])
                    append!(loc₂,loc[i][2])
                    append!(loc₃,loc[i][3])
                end

                # PyPlot.imsave("PQEpoch/1"*string(i)*"/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png",moo,vmin=0,vmax = 1,cmap = "gray")

                f_image = FFTW.fft(moo)
                f_images = (abs.(f_image)).^2
                f_images = collect(Iterators.flatten(f_images))

                Abin,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                Abin = π* Abin.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                # plotd = Plots.plot(kvals,Abin,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e1,1e8))
                # Plots.savefig(plotd,"PQEpoch/Fourier/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png")

                append!(saxion,[Abin])

                f_image = FFTW.fft(angle)
                f_images = (abs.(f_image)).^2
                f_images = collect(Iterators.flatten(f_images))

                Abins,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                Abins = π* Abins.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                # plotc = Plots.plot(kvals,Abins,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e6,1e11))
                # Plots.savefig(plotc,"PQEpoch/Angle/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png")


                append!(axion,[Abins])




                f_image = FFTW.fft( ( (Ȧ₂ .*  A₁ .- Ȧ₁ .* A₂)./(A₁.^2 .+ A₂.^2) ).^2 .* fₐ^2)
                f_images = (abs.(f_image)).^2
                f_images = collect(Iterators.flatten(f_images))

                Abint,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                Abint = π* Abint.* (kbins[2:end].^2 - kbins[1:end-1].^2)


                append!(axenergy,[Abint])

                ploot = Plots.plot(loc₁,loc₂,loc₃,seriestype = :scatter,xlims = (0,N),ylims = (0,N),zlims = (0,N),legend = false)
                Plots.savefig(ploot,"3D/PQTOWN/"*string(i)*"/"*lpad( string(trunc(Int, lo/x )) ,3,"0")*".png")
                
            end
            
            if tim < 250
                PQupdate_3D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt,tim,fₐ)
                tim = tim + Δt
            else
                PQupdate_3D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt * 250/tim,tim,fₐ)
                tim = tim + Δt * 250/tim
            end
            lo += 1
        end
     


        save("3D/PQTOWN/Saving/"*string(i)*"/Box"*string(n)*string(🥪)*".jld", "Real", A₁,"Imaginary",A₂,"Realvel",Ȧ₁,"RealIm",Ȧ₂)




        save("3D/PQTOWN/Saving/"*string(i)*"/Strings"*string(n)*string(🥪)*".jld","time",tracker,"number",cores)
        save("3D/PQTOWN/Saving/"*string(i)*"/FFT"*string(n)*string(🥪)*".jld","saxion",saxion,"axion",axion,"axenergy",axenergy)



    end



    return nothing
end

FPQrun_3D! (generic function with 1 method)

In [7]:
function FErun_3D!(n,N,t₁,A₁,A₂,Ȧ₁,Ȧ₂,Δx,Δt,fₐ,i,r,s,repeats)


    

    M₁ = zeros(N,N,N);
    M₂ = zeros(N,N,N);

    F₁ = zeros(N,N,N);
    F₂ = zeros(N,N,N);

    Laplacian_3D!(M₁,M₂,A₁,A₂,Δx)

    moo = zeros(N,N,N);
    angle = zeros(N,N,N);
    angler!(angle,A₁,A₂);

    k_freq = fftfreq(N)*N
    kx,ky,kz = meshgrid(k_freq,k_freq,k_freq)

    knrm = sqrt.( kx.^2 + ky.^2 + kz.^2)
    knrm = collect(Iterators.flatten(knrm))

    kbins = range(0.5, N/2+1, step = 1)
    kvals = 0.5 * (kbins[2:end] + kbins[1:end-1])
    
    x = 3

    tracker = []
    cores = []
    thr = 1
    lo = 0
    strng = 1
    mng = 0

    saxion = []
    axion = []
    axenergy = []
    
    
    Ȧ = zeros(N,N,N)

    Ȧ .= (Ȧ₂ .*  A₁ .- Ȧ₁ .* A₂)./(A₁.^2 .+ A₂.^2)

    M = zeros(N,N,N);

    F = zeros(N,N,N);

    LLaplacian_3D!(M,angle,Δx)

    tim = t₁
    
    for 🥪 ∈ 1:repeats
        
        

        while tim < 0.5*🥪 + t₁
        #while strng != 0
            
            if strng != 0

                tim = round(tim,digits = 3);
                if lo % x == 0
                    #18 pictures
                    mooing!(moo,A₁,A₂);
                    #println(moo[1,1])
                    #setting!(moo);
                    angler!(angle,A₁,A₂);
                    
                    

                    strng,loc = Cores_3D!(N,angle,thr)
                    
                    loc₁ = []
                    loc₂ = []
                    loc₃ = []

                    for i ∈ 1:length(loc)
                        append!(loc₁,loc[i][1])
                        append!(loc₂,loc[i][2])
                        append!(loc₃,loc[i][3])
                    end
          
                    
                    append!(tracker,tim)
                    append!(cores,strng)

                    # PyPlot.imsave("EQCD/1"*string(i)*"/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png",moo,vmin=0,vmax = 1,cmap = "gray")


                    f_image = FFTW.fft(moo)
                    f_images = (abs.(f_image)).^2
                    f_images = collect(Iterators.flatten(f_images))

                    Abin,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    Abin = π* Abin.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                    # plotd = Plots.plot(kvals,Abin,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e1,1e8))
                    # Plots.savefig(plotd,"EQCD/Fourier/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png")

                    append!(saxion,[Abin])

                    f_image = FFTW.fft(angle)
                    f_images = (abs.(f_image)).^2
                    f_images = collect(Iterators.flatten(f_images))

                    Abins,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    Abins = π* Abins.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                    # plotc = Plots.plot(kvals,Abins,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e6,1e11))
                    # Plots.savefig(plotc,"EQCD/Angle/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png")

                    append!(axion,[Abins])


                    f_image = FFTW.fft( ( (Ȧ₂ .*  A₁ .- Ȧ₁ .* A₂)./(A₁.^2 .+ A₂.^2) ).^2 .* fₐ^2)
                    f_images = (abs.(f_image)).^2
                    f_images = collect(Iterators.flatten(f_images))

                    Abint,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    Abint = π* Abint.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                    append!(axenergy,[Abint])



                    
                    if r == 1 && s == 1
                        ploot = Plots.plot(loc₁,loc₂,loc₃,seriestype = :scatter,xlims = (0,N),ylims = (0,N),zlims = (0,N),legend = false)
                        Plots.savefig(ploot,"3D/QCDTOWN/"*string(i)*"/"*lpad( string(trunc(Int, lo/x )) ,3,"0")*".png")
                    end



                end
                if tim < 1.8
                
                    EQCDupdate_3D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt,tim,fₐ,r,s)
                    tim = tim + Δt
                    
                else
                    EQCDupdate_3D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt*(1.8/tim)^3.34,tim,fₐ,r,s)
                    tim = tim + Δt*(1.8/tim)^3.34
                    
                end
                lo += 1
                
                


                
            else
                
                if mng == 0
                    Ȧ = zeros(N,N,N)

                    Ȧ .= (Ȧ₂ .*  A₁ .- Ȧ₁ .* A₂)./(A₁.^2 .+ A₂.^2)

                    M = zeros(N,N,N);

                    F = zeros(N,N,N);

                    LLaplacian_3D!(M,angle,Δx)
                end


                tim = round(tim,digits = 3);
                if lo % x == 0
                    #13 pictures
                    #mooing!(moo,A₁,A₂);
                    #println(moo[1,1])
                    #setting!(moo);
                    #angler!(angle,A₁,A₂);

                    strng,loc = Cores_3D!(N,angle,thr)

                    

                    append!(tracker,tim)
                    append!(cores,strng)

                    # PyPlot.imsave("EQCD/1"*string(i)*"/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png",moo,vmin=0,vmax = 1,cmap = "gray")

                    # f_image = FFTW.fft(moo)
                    # f_images = (abs.(f_image)).^2
                    # f_images = collect(Iterators.flatten(f_images))

                    # Abin,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    # Abin = π* Abin.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                    # plotd = Plots.plot(kvals,Abin,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e1,1e8))
                    # Plots.savefig(plotd,"EQCD/Fourier/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png")

                    #append!(saxion,[Abin])

                    f_image = FFTW.fft(angle)
                    f_images = (abs.(f_image)).^2
                    f_images = collect(Iterators.flatten(f_images))

                    Abins,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    Abins = π* Abins.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                    # plotc = Plots.plot(kvals,Abins,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e6,1e11))
                    # Plots.savefig(plotc,"EQCD/Angle/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png")

                    append!(axion,[Abins])


                    f_image = FFTW.fft( Ȧ.^2 .* fₐ^2)
                    f_images = (abs.(f_image)).^2
                    f_images = collect(Iterators.flatten(f_images))

                    Abint,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    Abint = π* Abint.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                    append!(axenergy,[Abint])






#                     if r == 1 && s == 1

#                         PyPlot.imsave("3D/QCDTOWN/"*string(i)*"/"*lpad( string(trunc(Int, lo/x )) ,3,"0")*".png",angle,vmin=-π,vmax = π,cmap = "twilight")
#                     end

                    


                end
                
                if tim < 1.8
                    Lupdate_3D!(angle,Ȧ,M,F,Δx,Δt,tim,fₐ,s)
                    tim = tim + Δt
                else
                    Lupdate_3D!(angle,Ȧ,M,F,Δx,Δt*(1.8/tim)^3.34,tim,fₐ,s)
                    tim = tim + Δt*(1.8/tim)^3.34
                end
                
                lo += 1
   
                mng = 1

               
            end
           

        end

        save("3D/QCDTOWN/Saving/"*string(i)*"/Box"*string(n)*string(🥪)*".jld", "Real", A₁,"Imaginary",A₂,"Realvel",Ȧ₁,"RealIm",Ȧ₂,"Angle",angle,"AngVel",Ȧ)



        save("3D/QCDTOWN/Saving/"*string(i)*"/Strings"*string(n)*string(🥪)*".jld","time",tracker,"number",cores)
        save("3D/QCDTOWN/Saving/"*string(i)*"/FFT"*string(n)*string(🥪)*".jld","saxion",saxion,"axion",axion,"axenergy",axenergy)



 

        
    end


    

    return nothing
end

FErun_3D! (generic function with 1 method)

In [8]:
# using Pkg
# ENV["PYTHON"]="/usr/bin/python3.8"
# Pkg.build("PyCall")


# using CircularArrays
# using JLD
# using FFTW
# #Pkg.add("PyCall")
# #Pkg.build("PyCall")
# #Pkg.add("SciPy")
# #Pkg.add("Conda")
# using Conda
# Conda.add("scipy")
# using PyCall
# stats = pyimport("scipy.stats")

In [ ]:
@time begin


    
n = 5
N = 2^n;
println(N)

rₐ = 15
fₐ = 10 .^rₐ;
r = 1
s = 1
    
    
    
LPQ = [125 250 500 1000 2000 4000 8000 16000 32000 64000]
LQCD = LPQ/600*0.4


for i ∈ 3:3

#     t₀ = 0.0001;
#     print(string(i)*" ")


#     Ȧ₁ = zeros(N,N,N);
#     Ȧ₂ = zeros(N,N,N);


#     μ,σ = 0, 0.1;
#     C₁ = rand(Normal(μ,σ),N,N,N);
#     C₂ = rand(Normal(μ,σ),N,N,N);


#     Δx = round(LPQ[i]/N,digits = 4)
#     Δt = 0.004
        
#     repeat = 3
        

#     FPQrun_3D!(n, N, t₀, C₁, C₂, Ȧ₁, Ȧ₂, Δx, Δt,fₐ,i,repeat)
        
    repeat = 3
        
        
    C₁ = load("3D/PQTOWN/Saving/"*string(i)*"/Box"*string(n)*string(repeat)*".jld")["Real"]
    C₂ = load("3D/PQTOWN/Saving/"*string(i)*"/Box"*string(n)*string(repeat)*".jld")["Imaginary"]
    Ȧ₁ = load("3D/PQTOWN/Saving/"*string(i)*"/Box"*string(n)*string(repeat)*".jld")["Realvel"]
    Ȧ₂ = load("3D/PQTOWN/Saving/"*string(i)*"/Box"*string(n)*string(repeat)*".jld")["RealIm"]
    
    t₁ = 0.4
        
    Δx = round(LQCD[i]/N,digits = 6)
    Δt = 0.001
        
    repeat = 5
        
    FErun_3D!(n,N,t₁,C₁,C₂,Ȧ₁,Ȧ₂,Δx,Δt,fₐ,i,r,s,repeat)


end
    
    

for i ∈ 7:7

#     t₀ = 0.0001;
#     print(string(i)*" ")


#     Ȧ₁ = zeros(N,N,N);
#     Ȧ₂ = zeros(N,N,N);


#     μ,σ = 0, 0.1;
#     C₁ = rand(Normal(μ,σ),N,N,N);
#     C₂ = rand(Normal(μ,σ),N,N,N);


#     Δx = round(LPQ[i]/N,digits = 4)
#     Δt = 0.004
        
#     repeat = 3
        

#     FPQrun_3D!(n, N, t₀, C₁, C₂, Ȧ₁, Ȧ₂, Δx, Δt,fₐ,i,repeat)
        
#     repeat = 3
        
        
    C₁ = load("3D/PQTOWN/Saving/"*string(i)*"/Box"*string(n)*string(repeat)*".jld")["Real"]
    C₂ = load("3D/PQTOWN/Saving/"*string(i)*"/Box"*string(n)*string(repeat)*".jld")["Imaginary"]
    Ȧ₁ = load("3D/PQTOWN/Saving/"*string(i)*"/Box"*string(n)*string(repeat)*".jld")["Realvel"]
    Ȧ₂ = load("3D/PQTOWN/Saving/"*string(i)*"/Box"*string(n)*string(repeat)*".jld")["RealIm"]
    
    t₁ = 0.4
        
    Δx = round(LQCD[i]/N,digits = 6)
    Δt = 0.001
        
    repeat = 5
        
    FErun_3D!(n,N,t₁,C₁,C₂,Ȧ₁,Ȧ₂,Δx,Δt,fₐ,i,r,s,repeat)


end
    
    

    println(' ')
end

32


In [10]:
loc = [[1,1,1],[2,3,2],[3,3,3],[4,4,4],[5,5,5]]
loc₁ = []

for i ∈ 1:length(loc)
    append!(loc₁,loc[i][1])
end
print(loc₁)
#Plots.scatter(loc,seriestype = :scatter,xlims = (0,5),ylims = (0,5),zlims = (0,5),legend = false)

Any[1, 2, 3, 4, 5]

In [16]:

#load("3D/PQTOWN/Saving/3/Strings53.jld")["time"]
load("3D/QCDTOWN/Saving/3/Strings53.jld")["time"]

1001-element Vector{Any}:
 0.4
 0.403
 0.406
 0.409
 0.412
 0.415
 0.418
 0.421
 0.424
 0.427
 0.43
 0.433
 0.436
 ⋮
 1.867
 1.87
 1.873
 1.876
 1.879
 1.882
 1.885
 1.888
 1.891
 1.894
 1.897
 1.9

In [ ]:

saxion = load("3D/PQTOWN/Saving/3/FFT53.jld")["saxion"]
axion = load("3D/PQTOWN/Saving/3/FFT53.jld")["axion"]
axenergy = load("3D/PQTOWN/Saving/3/FFT53.jld")["axenergy"]
tracker = load("3D/PQTOWN/Saving/3/Strings53.jld")["time"]
cores = load("3D/PQTOWN/Saving/3/Strings53.jld")["number"]

n = 5

N = 2^n

k_freq = fftfreq(N)*N
kx,ky,kz = meshgrid(k_freq,k_freq,k_freq)

knrm = sqrt.( kx.^2 + ky.^2 + kz.^2)
knrm = collect(Iterators.flatten(knrm))

kbins = range(0.5, N/2+1, step = 1)
kvals = 0.5 * (kbins[2:end] + kbins[1:end-1])

lowₛ = minimum(minimum(saxion))
highₛ = maximum(maximum(saxion))
lowₐ = minimum(minimum(axion))
highₐ = maximum(maximum(axion))
lowₑ = minimum(minimum(axenergy))
highₑ = maximum(maximum(axenergy))

for i ∈ 1:length(tracker)

plotc = Plots.plot(kvals,saxion[i],xaxis =:log ,yaxis =:log,legend = false, ylims = (lowₛ,highₛ),title = "Saxion η = "*string(tracker[i]))
Plots.savefig(plotc,"3D/PQTOWN/Saving/3/Saxion/"*lpad( string(i-1) ,3,"0")*".png")


plotd = Plots.plot(kvals,axion[i],xaxis =:log ,yaxis =:log,legend = false, ylims = (lowₐ,highₐ),title = "Axion (a/fₐ) η = "*string(tracker[i]))
Plots.savefig(plotd,"3D/PQTOWN/Saving/3/Axion/"*lpad( string(i-1) ,3,"0")*".png")


plote = Plots.plot(kvals,axenergy[i]/1e30,xaxis =:log ,yaxis = :log,title = "Axion Energy Density (ρₐ = ȧ^2 / fₐ^2) η = "*string(tracker[i]),legend = false,ylims = (lowₑ,highₑ))
Plots.savefig(plote,"3D/PQTOWN/Saving/3/Axenergy/"*lpad( string(i-1) ,3,"0")*".png")

end


plotf = Plots.plot(tracker,cores,title = "String Count",legend = false,xaxis =:log ,yaxis = :log)
Plots.savefig(plotf,"3D/PQTOWN/Saving/3/Strings.png")


In [ ]:

saxion = load("3D/PQTOWN/Saving/7/FFT53.jld")["saxion"]
axion = load("3D/PQTOWN/Saving/7/FFT53.jld")["axion"]
axenergy = load("3D/PQTOWN/Saving/7/FFT53.jld")["axenergy"]
tracker = load("3D/PQTOWN/Saving/7/Strings53.jld")["time"]
cores = load("3D/PQTOWN/Saving/7/Strings53.jld")["number"]

n = 5

N = 2^n

k_freq = fftfreq(N)*N
kx,ky,kz = meshgrid(k_freq,k_freq,k_freq)

knrm = sqrt.( kx.^2 + ky.^2 + kz.^2)
knrm = collect(Iterators.flatten(knrm))

kbins = range(0.5, N/2+1, step = 1)
kvals = 0.5 * (kbins[2:end] + kbins[1:end-1])

lowₛ = minimum(minimum(saxion))
highₛ = maximum(maximum(saxion))
lowₐ = minimum(minimum(axion))
highₐ = maximum(maximum(axion))
lowₑ = minimum(minimum(axenergy))
highₑ = maximum(maximum(axenergy))

for i ∈ 1:length(tracker)

plotc = Plots.plot(kvals,saxion[i],xaxis =:log ,yaxis =:log,legend = false, ylims = (lowₛ,highₛ),title = "Saxion η = "*string(tracker[i]))
Plots.savefig(plotc,"3D/PQTOWN/Saving/7/Saxion/"*lpad( string(i-1) ,3,"0")*".png")


plotd = Plots.plot(kvals,axion[i],xaxis =:log ,yaxis =:log,legend = false, ylims = (lowₐ,highₐ),title = "Axion (a/fₐ) η = "*string(tracker[i]))
Plots.savefig(plotd,"3D/PQTOWN/Saving/7/Axion/"*lpad( string(i-1) ,3,"0")*".png")


plote = Plots.plot(kvals,axenergy[i]/1e30,xaxis =:log ,yaxis = :log,title = "Axion Energy Density (ρₐ = ȧ^2 / fₐ^2) η = "*string(tracker[i]),legend = false,ylims = (lowₑ,highₑ))
Plots.savefig(plote,"3D/PQTOWN/Saving/7/Axenergy/"*lpad( string(i-1) ,3,"0")*".png")

end


plotf = Plots.plot(tracker,cores,title = "String Count",legend = false,xaxis =:log ,yaxis = :log)
Plots.savefig(plotf,"3D/PQTOWN/Saving/7/Strings.png")


In [ ]:
saxion = load("3D/QCDTOWN/Saving/3/FFT55.jld")["saxion"]
axion = load("3D/QCDTOWN/Saving/3/FFT55.jld")["axion"]
axenergy = load("3D/QCDTOWN/Saving/3/FFT55.jld")["axenergy"]
tracker = load("3D/QCDTOWN/Saving/3/Strings55.jld")["time"]
cores = load("3D/QCDTOWN/Saving/3/Strings55.jld")["number"]

lowₛ = minimum(minimum(saxion))
highₛ = maximum(maximum(saxion))
lowₐ = minimum(minimum(axion))
highₐ = maximum(maximum(axion))
lowₑ = minimum(minimum(axenergy))
highₑ = maximum(maximum(axenergy))

for i ∈ 1:length(saxion)

    plotc = Plots.plot(kvals,saxion[i],xaxis =:log ,yaxis =:log,legend = false, ylims = (lowₛ,highₛ),title = "λ = 5504 ,ηᵪ = 2.8 ,Saxion η = "*string(tracker[i]))
    Plots.savefig(plotc,"3D/QCDTOWN/Saving/3/Saxion/"*lpad( string(i-1) ,3,"0")*".png")
    
end

for i ∈ 1:length(axion)

    plotd = Plots.plot(kvals,axion[i],xaxis =:log ,yaxis =:log,legend = false, ylims = (lowₐ,highₐ),title = "λ = 5504 ,ηᵪ = 2.8 ,Axion (a/fₐ) η = "*string(tracker[i]))
    Plots.savefig(plotd,"3D/QCDTOWN/Saving/3/Axion/"*lpad( string(i-1) ,3,"0")*".png")

end

for i ∈ 1:length(axenergy)

    plote = Plots.plot(kvals,axenergy[i]/1e30,xaxis =:log ,yaxis = :log,title = "Axion Energy Density (ρₐ = ȧ^2 / fₐ^2) η = "*string(tracker[i]),legend = false,ylims = (lowₑ,highₑ))
    Plots.savefig(plote,"3D/QCDTOWN/Saving/3/Axenergy/"*lpad( string(i-1) ,3,"0")*".png")

end



plotf = Plots.plot(tracker,cores,title = "λ = 5504 ,ηᵪ = 2.8 ,tring Count",legend = false)
Plots.savefig(plotf,"3D/QCDTOWN/Saving/3/Strings.png")


In [ ]:
saxion = load("3D/QCDTOWN/Saving/7/FFT55.jld")["saxion"]
axion = load("3D/QCDTOWN/Saving/7/FFT55.jld")["axion"]
axenergy = load("3D/QCDTOWN/Saving/7/FFT55.jld")["axenergy"]
tracker = load("3D/QCDTOWN/Saving/7/Strings55.jld")["time"]
cores = load("3D/QCDTOWN/Saving/7/Strings55.jld")["number"]

lowₛ = minimum(minimum(saxion))
highₛ = maximum(maximum(saxion))
lowₐ = minimum(minimum(axion))
highₐ = maximum(maximum(axion))
lowₑ = minimum(minimum(axenergy))
highₑ = maximum(maximum(axenergy))

for i ∈ 1:length(saxion)

    plotc = Plots.plot(kvals,saxion[i],xaxis =:log ,yaxis =:log,legend = false, ylims = (lowₛ,highₛ),title = "λ = 5504 ,ηᵪ = 2.8 ,Saxion η = "*string(tracker[i]))
    Plots.savefig(plotc,"3D/QCDTOWN/Saving/7/Saxion/"*lpad( string(i-1) ,3,"0")*".png")
    
end

for i ∈ 1:length(axion)

    plotd = Plots.plot(kvals,axion[i],xaxis =:log ,yaxis =:log,legend = false, ylims = (lowₐ,highₐ),title = "λ = 5504 ,ηᵪ = 2.8 ,Axion (a/fₐ) η = "*string(tracker[i]))
    Plots.savefig(plotd,"3D/QCDTOWN/Saving/7/Axion/"*lpad( string(i-1) ,3,"0")*".png")

end

for i ∈ 1:length(axenergy)

    plote = Plots.plot(kvals,axenergy[i]/1e30,xaxis =:log ,yaxis = :log,title = "Axion Energy Density (ρₐ = ȧ^2 / fₐ^2) η = "*string(tracker[i]),legend = false,ylims = (lowₑ,highₑ))
    Plots.savefig(plote,"3D/QCDTOWN/Saving/7/Axenergy/"*lpad( string(i-1) ,3,"0")*".png")

end



plotf = Plots.plot(tracker,cores,title = "λ = 5504 ,ηᵪ = 2.8 ,tring Count",legend = false)
Plots.savefig(plotf,"3D/QCDTOWN/Saving/7/Strings.png")
